In [207]:
import mido
import sys
import binascii
import pprint
from collections import OrderedDict
from mido import Message, MidiFile, MidiTrack

In [38]:
'''
C C# D D# E F F# G G# A A# B
'''
NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
MAP_DEC_TO_NOTE = {}
MAP_NOTE_TO_DEC = {}
octave = -1
i = 0
j = 0
while i<128:
    MAP_DEC_TO_NOTE[ i ] = NOTES[j] + '_' + str(octave)
    MAP_NOTE_TO_DEC[ NOTES[j] + '_' + str(octave) ] = i
    if j == len(NOTES)-1:
        j = 0
        octave += 1
    else:
        j += 1
    i += 1

In [384]:
mid = mido.MidiFile('beethoven.mid')

In [392]:
#for i, track in enumerate(mid.tracks):
track = mid.tracks[2]
print('Track {}: {}'.format(i, track.name))
#for msg in track:
#    print(msg)

Track 7: Sonata Quasi Una Fantasia 


In [116]:
'''
Message attributes:
- type
- note
- velocity
- time
'''
'''
i=0
for msg in mid.play():
    if msg.type == 'note_on':
        print(f'{msg.type} / { MAP_DEC_TO_NOTE[msg.note] } / {msg.time}')
        i += 1
        if i>30:
            break
'''

"\ni=0\nfor msg in mid.play():\n    if msg.type == 'note_on':\n        print(f'{msg.type} / { MAP_DEC_TO_NOTE[msg.note] } / {msg.time}')\n        i += 1\n        if i>30:\n            break\n"

---

In [45]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

BEAT = 96
DEC = MAP_NOTE_TO_DEC[ 'G_4' ]

track.append(Message('program_change', program=12, time=0))
track.append(Message('note_on', note=MAP_NOTE_TO_DEC[ 'G_4' ], velocity=64, time=BEAT))
track.append(Message('note_off', note=MAP_NOTE_TO_DEC[ 'G_4' ], velocity=64, time=BEAT))

track.append(Message('note_on', note=MAP_NOTE_TO_DEC[ 'A_4' ], velocity=64, time=BEAT))
track.append(Message('note_off', note=MAP_NOTE_TO_DEC[ 'A_4' ], velocity=64, time=BEAT))

track.append(Message('note_on', note=MAP_NOTE_TO_DEC[ 'B_4' ], velocity=64, time=BEAT))
track.append(Message('note_off', note=MAP_NOTE_TO_DEC[ 'B_4' ], velocity=64, time=BEAT))

mid.save('new_song.mid')

---

#### Goal:
- Python script injest .mid, parse .mid, generate .cairo
- The generated Cairo contract should have @view functions to inject "interpretation" and return interpreted .mid file
    - what constitutes interpretation?
    - change overall tempo - USE TEMPO TRACK
    - change velocity (note strength)
    - advanced: rubato -- require "musical phrase" baked in data structure; ALSO USE TEMPO TRACK

In [281]:
with open('beethoven.mid', 'rb') as f:
    # Slurp the whole file and efficiently convert it to hex all at once
    hexdata = binascii.hexlify(f.read())

In [282]:
type(hexdata)

bytes

- 4d546864 -- MThd
- 00000006 -- 6 bytes follow
- 0001 -- format 1
- 0008 -- number of tracks
- 0078 -- tickdiv = 120 ppqn
- b'4d54686400000006000100080078'

In [78]:
#byte_val = b'\x4D\x54\x68\x64' # 4D 54 68 64
'''
>>> b'\xde\xad\xbe\xef'.hex()
'deadbeef'
and reverse:

>>> bytes.fromhex('deadbeef')
b'\xde\xad\xbe\xef'
'''
byte_val = b'MThd'
int_val = int.from_bytes(byte_val, 'big')
int_val

1297377380

In [77]:
int_val.to_bytes(4, byteorder='big')

b'MThd'

In [258]:
class TempoTrack:
    def __init__(self, track):
        self.track = track
        self.segs = {}
        self._initialize()
        
    def _initialize(self):
        segs = {}
        for i,seg in enumerate(self.track.split('ff')):
            if i==0:
                segs['header'] = seg
            else:
                seg_ = 'ff'+seg
                if seg_[0:6] != 'ff5103':
                    segs[f'meta-event-{i}'] = (seg_)
                    continue
                if len(seg_) > 12:
                    segs[f'temp-event-{i}'] = {
                        'event' : seg_[0:6],
                        'value' : seg_[6:12]
                    }
                    segs[f'delta-time-{i}'] = seg_[12:]
        self.segs = segs
    
    def print_segs(self):
        print(self.segs)
        
    def hexstring(self):
        hexstring = ''
        for k,v in self.segs.items():
            if type(v) == str:
                hexstring += v
            else:
                for kk,vv in v.items():
                    hexstring += vv
        return hexstring
        
    def div_tempo_value(self, divider):
        assert divider != 0
        for k,v in self.segs.items():
            if type(v) == str:
                continue
            assert v['event'] == 'ff5103'
            value_int = int(v['value'], 16)
            value_int_divided = int(value_int/divider)
            value_hex_divided = hex(value_int_divided)[2:]
            value_hex_divided = value_hex_divided.rjust(6, '0')
            v['value'] = value_hex_divided

In [358]:
hexdata_str = hexdata.decode("utf-8")
'''
4D 54 68 64 = MThd
4D 54 72 6B = MTrk
FF 03 = Meta-Event: Sequence/Track Name
FF 7F = Meta-Event: Sequencer Specifics
FF 58 04 = Meta-Event: Time Signature
FF 59 02 = Meta-Event: Key Signature
FF 51 03 = Meta-Event: Set Tempo
FF 2F 00 = Meta-Event: End of Track
'''

MTrk = '4d54726b'
tracks = []
for i,chunk in enumerate(hexdata_str.split(MTrk)):
    if i == 0:
        header = chunk
    else:
        tracks.append (MTrk + chunk)

tempo_track = TempoTrack(tracks[0])
#tempo_track.div_tempo_value(divider=2.5)

---

### Surgery on the midi file - scale up/down overall tempo

In [353]:
## Reduce tempo track tttttt value (microseconds per quarter note)
tempo_track_str = tempo_track.hexstring()

In [354]:
## Changed header to indicate 3 tracks only
header_trimmed = '4d54686400000006000100030078' # 3 tracks

## Take only 3 tracks (the last 5 of the original 8 tracks are not useful)
hexdata_str_trimmed = header_trimmed + ''.join( [tempo_track_str, tracks[1], tracks[2]] )
#hexdata_str_trimmed = header_trimmed + ''.join( tracks[0:3] )

with open('beethoven_recovered.mid', 'wb') as fout:
    fout.write( binascii.unhexlify(hexdata_str_trimmed) )

### Conceptualizing Cairo contract structure
- Define: `midi_felt_array` as an array of felts to be assembled into .mid by python frontend
- Function-wise. the contract exposes the following functions:
    - a view function which, upon calling, returns `midi_felt_array`
    - a view function that takes a `divider` value as input (used to scale up/down tempo), and returns `midi_felt_array`
- question: how to 'standardize' the felt representation of hex strings?
    - largest positive value is a 63-digit hex
    - let's use 62-digit hex i.e. each felt represents up to 62 hex values (FF.....F, len=62)

In [355]:
def hexstring_into_felt_array (hexstring):
    '''
    last element of the felt array is an integer indicating the hex-length of the last felt value
    '''
    arr = []
    s = hexstring
    while( len(s)>62 ):
        felt_hex = s[0:62]
        arr.append( int(felt_hex, 16) )
        s = s[62:]
    felt_hex = s
    last_length = len(felt_hex)
    arr.append( int(felt_hex, 16) )
    arr.append(last_length)
    return arr

def felt_array_into_hexstring (felt_array):
    hexstring = ''
    last_length = felt_array[-1]
    for felt in felt_array[:-2]:
        hexstr = hex(felt)[2:]
        hexstr = hexstr.rjust(62, '0')
        hexstring += hexstr
    hexstr = hex(felt_array[-2])[2:]
    hexstr = hexstr.rjust(last_length, '0')
    hexstring += hexstr
    return hexstring

In [445]:
def tempo_track_into_assertions_tempo_adjustable (tempo_track):
    d = tempo_track.segs
    i = 0
    for k,v in d.items():
        if type(v)==str:
            integer = int(v, 16)
            length = len(v)
            print(f'    assert [z+{i}] = {integer}')
            print(f'    assert [z+{i+1}] = {length}')
            i += 2
        else:
            event_integer = int(v['event'], 16)
            event_length  = len(v['event'])
            value_integer = int(v['value'], 16)
            value_length  = len(v['value'])
            assert value_length == 6
            print(f'\n    # Set Tempo at adjusted value')
            print(f'    assert [z+{i}] = {event_integer}')
            print(f'    assert [z+{i+1}] = {event_length}')
            print(f'    tempvar value_ = {value_integer} * tempo_multiplier')
            print(f'    let (adjusted_value, _) = unsigned_div_rem(value_, tempo_divider)')
            print(f'    assert [z+{i+2}] = adjusted_value')
            print(f'    assert [z+{i+3}] = 6') # the hex-length of tempo value is always 6
            i += 4
    return

def hybrid_array_into_hexstring (hybrid_array):
    assert len(hybrid_array) % 2 == 0 # length is even-number
    hexstring = ''
    i=0
    while i<len(hybrid_array):
        value = hybrid_array[i]
        length = hybrid_array[i+1]
        hexstr = hex(value)[2:]
        hexstr = hexstr.rjust(length, '0')
        hexstring += hexstr
        i += 2
    return hexstring

#tempo_track_into_assertions_tempo_adjustable(tempo_track)

In [316]:
'''
PRIME = 3618502788666131213697322783095070105623107215331596699973092056135872020481
PRIME_HALF = PRIME//2 # largest positive number in decimal

LARGEST_POS_HEX = hex(PRIME_HALF)[2:]
print(LARGEST_POS_HEX) # 400000000000008800000000000000000000000000000000000000000000000
print(f'len = {len(LARGEST_POS_HEX)}') # 63
'''

"\nPRIME = 3618502788666131213697322783095070105623107215331596699973092056135872020481\nPRIME_HALF = PRIME//2 # largest positive number in decimal\n\nLARGEST_POS_HEX = hex(PRIME_HALF)[2:]\nprint(LARGEST_POS_HEX) # 400000000000008800000000000000000000000000000000000000000000000\nprint(f'len = {len(LARGEST_POS_HEX)}') # 63\n"

```
func midi_felt_array {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z] = ...
    assert [z+..] = ...
    ...
    
    let z_len = ...

    return (z_len, z)
end
```

In [347]:
header_trimmed = '4d54686400000006000100030078' # 3 tracks
arr_header = hexstring_into_felt_array(header_trimmed)

arr_tempo = hexstring_into_felt_array( tempo_track.hexstring() )

arr_track1 = hexstring_into_felt_array(tracks[1])

arr_track2 = hexstring_into_felt_array(tracks[2])

arr_s = [arr_header, arr_tempo, arr_track1, arr_track2]

In [348]:
for arr in arr_s:
    i = 0
    for val in arr:
        print(f'    assert [z+{i}] = {val}')
        i += 1
    print(f'    length = {len(arr)}')
    print()

    assert [z+0] = 1568433012465980210040715044126840
    assert [z+1] = 28
    length = 2

    assert [z+0] = 136630055383401276312354875491708806028802975461908127728039954538714585092
    assert [z+1] = 7081839633435791322100975806561433991310438160632423037906578163376523299
    assert [z+2] = 150464953677976688370044382826598040127303780180748866245826816664452735829
    assert [z+3] = 72436011984389210998808387443191368157341262216884305852579204620055958824
    assert [z+4] = 451105124672982480761980777849504643738880863339190071926657736227800447231
    assert [z+5] = 143135547091350967275422621001705966206121978189407921608921359239504528135
    assert [z+6] = 146653218246920417395770770845223310003379652949816700158895527202836907859
    assert [z+7] = 912351483177090142040302798520885290190758891315099175257724049945723013
    assert [z+8] = 213783777166593181357825727611898105032207163960639023956099816617850502993
    assert [z+9] = 5356707767972443758945661460773290544911

---

In [449]:
with open('./chopin_etude4.mid', 'rb') as f:
    hexdata = binascii.hexlify(f.read())
hexdata_str = hexdata_c.decode("utf-8")

In [450]:
MTrk = '4d54726b'
tracks = []
for i,chunk in enumerate(hexdata_str.split(MTrk)):
    if i == 0:
        header = chunk
    else:
        tracks.append (MTrk + chunk)

In [451]:
header

'4d54686400000006000100030100'

In [438]:
### Testing changing tempo
tempo_track_c = TempoTrack(tracks_c[0])
tempo_track_c.div_tempo_value(divider=1.5)

hexstring_c_new = header_c_new + tempo_track_c.hexstring() + tracks_c[2]

with open('chopin_trimmed.mid', 'wb') as fout:
    fout.write( binascii.unhexlify(hexstring_c_new) )

In [447]:
header_c_new = '4d54686400000006000100020100' # 2 tracks

arr_header = hexstring_into_felt_array(header_c_new)

arr_track = hexstring_into_felt_array(tracks_c[2])

arr_s = [arr_header, arr_track]

In [448]:
for arr in arr_s:
    i = 0
    for val in arr:
        print(f'    assert [z+{i}] = {val}')
        i += 1
    print(f'    length = {len(arr)}')
    print()

    assert [z+0] = 1568433012465980210040715044061440
    assert [z+1] = 28
    length = 2

    assert [z+0] = 136630055383401698422926651574183410346914346988277994928405650950548656208
    assert [z+1] = 155486431121205454340997718604636506690432198040502336143382442881004355472
    assert [z+2] = 133065710037307056132114303946437113295626566154585310809835194216324792320
    assert [z+3] = 226460101183638113689112655529915456138278044941076887908447500423331334191
    assert [z+4] = 2652292619316168716106944994118683104942966320351396334811086796934320204
    assert [z+5] = 134290737441159531830775955289951012581148170150004530857535082148583718800
    assert [z+6] = 134784219597875447904471516561851928012441861895347338191703568536050073626
    assert [z+7] = 254945253018937112161944393431077079130038575239743040137328347569803250688
    assert [z+8] = 112307381881622604979976734208965406435112001834029315380611610316227313740
    assert [z+9] = 438700710884957525373122777386797353

In [446]:
tempo_track_into_assertions_tempo_adjustable(tempo_track_c)

    assert [z+0] = 1426484337369910878211
    assert [z+1] = 18

    # Set Tempo at adjusted value
    assert [z+2] = 16732419
    assert [z+3] = 6
    tempvar value_ = 298506 * tempo_multiplier
    let (adjusted_value, _) = unsigned_div_rem(value_, tempo_divider)
    assert [z+4] = adjusted_value
    assert [z+5] = 6
    assert [z+6] = 0
    assert [z+7] = 2
    assert [z+8] = 12147294557503857467666312554572555623165229526536444632259136870901008410369633369239958781073750873459680225647488346842983128270889894374912
    assert [z+9] = 118
    assert [z+10] = 168574917295526412905125953342547971129969753248795889052652267345444824437923889941313516550711066695448264747615381775155712
    assert [z+11] = 104
    assert [z+12] = 18399460692883671104
    assert [z+13] = 16

    # Set Tempo at adjusted value
    assert [z+14] = 16732419
    assert [z+15] = 6
    tempvar value_ = 300750 * tempo_multiplier
    let (adjusted_value, _) = unsigned_div_rem(value_, tempo_divider)
    assert [z+